# **Trabalho 3 - Reconhecimento de Padrões**
### Classificadores Bayesianos
##### Renan Henrique Cardoso - 379013

In [1]:
import pandas as pd
import numpy

derma = pd.read_csv("https://raw.githubusercontent.com/cardosorrenan/reconhecimentopadroes-ufc/master/datasets/dermatology.csv")
derma.columns = ['erythema', 'scaling', 'def_borders', 'itching', 'koebner_phen', 'poly_papules',
                 'fol_papules', 'oral_muc_inv', 'knee_elbow_inv', 'scalp_inv', 'family_history',
                 'melanin_incont', 'eos_infiltrate', 'pnl_infiltrate', 'fibrosis_papil_derm', 
                 'exocytosis', 'acanthosis', 'hyperkeratosis', 'parakeratosis', 'club_rete_ridges', 
                 'elong_rete_ridges', 'thinning_supra_epid', 'spongiform_pustule', 'munro_microabcess', 
                 'focal_hypergran', 'dis_granular_layer', 'vac_damage_basal', 'spongiosis', 'saw_tooth_retes', 
                 'fol_horn_plug', 'perifol_parakeratosis', 'inflam_mono_inf', 'band_like_inf', 'age', 'class']

derma['age'] = pd.to_numeric(derma['age'], errors='coerce').fillna(30)

In [2]:
derma.head()

,erythema,scaling,def_borders,itching,koebner_phen,poly_papules,fol_papules,oral_muc_inv,knee_elbow_inv,scalp_inv,family_history,melanin_incont,eos_infiltrate,pnl_infiltrate,fibrosis_papil_derm,exocytosis,acanthosis,hyperkeratosis,parakeratosis,club_rete_ridges,elong_rete_ridges,thinning_supra_epid,spongiform_pustule,munro_microabcess,focal_hypergran,dis_granular_layer,vac_damage_basal,spongiosis,saw_tooth_retes,fol_horn_plug,perifol_parakeratosis,inflam_mono_inf,band_like_inf,age,class
0,3,3,3,2,1,0,0,0,1,1,1,0,0,1,0,1,2,0,2,2,2,2,2,1,0,0,0,0,0,0,0,1,0,8.0,1
1,2,1,2,3,1,3,0,3,0,0,0,1,0,0,0,1,2,0,2,0,0,0,0,0,2,0,2,3,2,0,0,2,3,26.0,3
2,2,2,2,0,0,0,0,0,3,2,0,0,0,3,0,0,2,0,3,2,2,2,2,0,0,3,0,0,0,0,0,3,0,40.0,1
3,2,3,2,2,2,2,0,2,0,0,0,1,0,0,0,1,2,0,0,0,0,0,0,0,2,2,3,2,3,0,0,2,3,45.0,3
4,2,3,2,0,0,0,0,0,0,0,0,0,2,1,0,2,2,0,2,0,0,0,1,0,0,0,0,2,0,0,0,1,0,41.0,2


### **Análise inicial dos dados**

- Vetor médio

In [3]:
mean_vector = pd.DataFrame(derma.mean()).T
mean_vector

,erythema,scaling,def_borders,itching,koebner_phen,poly_papules,fol_papules,oral_muc_inv,knee_elbow_inv,scalp_inv,family_history,melanin_incont,eos_infiltrate,pnl_infiltrate,fibrosis_papil_derm,exocytosis,acanthosis,hyperkeratosis,parakeratosis,club_rete_ridges,elong_rete_ridges,thinning_supra_epid,spongiform_pustule,munro_microabcess,focal_hypergran,dis_granular_layer,vac_damage_basal,spongiosis,saw_tooth_retes,fol_horn_plug,perifol_parakeratosis,inflam_mono_inf,band_like_inf,age,class
0,2.068493,1.794521,1.553425,1.361644,0.635616,0.449315,0.167123,0.378082,0.613699,0.520548,0.126027,0.405479,0.139726,0.547945,0.336986,1.364384,1.956164,0.528767,1.293151,0.665753,0.994521,0.635616,0.29589,0.364384,0.394521,0.465753,0.457534,0.947945,0.454795,0.10411,0.115068,1.868493,0.556164,36.106849,2.805479


- Vetor de variâncias 

In [4]:
variance_vector = pd.DataFrame(derma.var()).T
variance_vector

,erythema,scaling,def_borders,itching,koebner_phen,poly_papules,fol_papules,oral_muc_inv,knee_elbow_inv,scalp_inv,family_history,melanin_incont,eos_infiltrate,pnl_infiltrate,fibrosis_papil_derm,exocytosis,acanthosis,hyperkeratosis,parakeratosis,club_rete_ridges,elong_rete_ridges,thinning_supra_epid,spongiform_pustule,munro_microabcess,focal_hypergran,dis_granular_layer,vac_damage_basal,spongiosis,saw_tooth_retes,fol_horn_plug,perifol_parakeratosis,inflam_mono_inf,band_like_inf,age,class
0,0.443098,0.493376,0.819253,1.291931,0.825651,0.91844,0.326389,0.69732,0.968493,0.821692,0.110447,0.758212,0.169983,0.665964,0.729535,1.215761,0.509062,0.574033,0.839651,1.118742,1.351618,1.072904,0.45067,0.578398,0.723047,0.749511,0.913714,1.269261,0.913473,0.203417,0.23947,0.526614,1.225546,230.20009,2.558212


- Matriz de covariância e matriz de correlação

In [30]:
def create_matrices():
  attrs = derma.columns.to_list()
  n_attrs = len(attrs)
  mtrx_cov = numpy.zeros(shape=(n_attrs, n_attrs))
  mtrx_corr = numpy.zeros(shape=(n_attrs, n_attrs))
  for i in range(0, n_attrs):
    for j in range(i, n_attrs):
      if (j != i):
        sum_attr1 = derma.apply(lambda row: row[attrs[i]] - mean_vector[attrs[i]], axis=1)
        sum_attr2 = derma.apply(lambda row: row[attrs[j]] - mean_vector[attrs[j]], axis=1)
        cov = (sum_attr1 * sum_attr2).sum() / (derma.shape[0] - 1)
        corr = cov / (derma[attrs[i]].std() * derma[attrs[j]].std())
        mtrx_cov[i,j] = mtrx_cov[j,i] = cov
        mtrx_corr[i,j] = mtrx_cov[i,j] = corr
      else:
        mtrx_cov[i,j] = mtrx_corr[i,j] = 1
  df_cov = pd.DataFrame(mtrx_cov)
  df_corr = pd.DataFrame(mtrx_corr)
  return df_cov, df_corr

%%time
cov, corr = create_matrices()

In [ ]:
derma.corr()

In [32]:
derma.cov()

,erythema,scaling,def_borders,itching,koebner_phen,poly_papules,fol_papules,oral_muc_inv,knee_elbow_inv,scalp_inv,family_history,melanin_incont,eos_infiltrate,pnl_infiltrate,fibrosis_papil_derm,exocytosis,acanthosis,hyperkeratosis,parakeratosis,club_rete_ridges,elong_rete_ridges,thinning_supra_epid,spongiform_pustule,munro_microabcess,focal_hypergran,dis_granular_layer,vac_damage_basal,spongiosis,saw_tooth_retes,fol_horn_plug,perifol_parakeratosis,inflam_mono_inf,band_like_inf,age,class
erythema,1.000000,0.435607,0.260654,-0.047353,0.000498,0.033449,-0.109647,-0.026946,0.141151,0.186609,0.171989,0.042008,0.075146,0.224048,-0.354787,0.022048,0.064184,-0.055667,0.210208,0.157469,0.007586,0.167784,0.169696,0.113366,-0.013896,0.159014,0.002425,0.023084,-0.005915,-0.005515,0.009474,0.075570,-0.007099,0.001450,-0.335800
scaling,0.435607,1.000000,0.360846,-0.085602,-0.010024,-0.074691,-0.092188,-0.082639,0.298182,0.302209,0.181852,-0.074514,0.042494,0.283231,-0.383395,-0.112344,0.108059,0.013719,0.298726,0.321456,0.139914,0.331044,0.298250,0.186830,-0.084684,0.212024,-0.109187,0.010748,-0.114134,-0.010335,-0.010949,-0.010041,-0.131738,0.021915,-0.468502
def_borders,0.260654,0.360846,1.000000,-0.056217,0.239187,0.326968,-0.168728,0.285790,0.305436,0.267366,0.114550,0.310555,-0.134167,0.038365,-0.277433,-0.199864,0.190815,0.040818,0.357020,0.386015,0.198695,0.368059,0.245189,0.241031,0.290224,0.353809,0.297138,-0.249164,0.273529,-0.087689,-0.125562,0.106925,0.281452,0.151721,-0.396057
itching,-0.047353,-0.085602,-0.056217,1.000000,0.271582,0.412835,-0.144099,0.362073,-0.302110,-0.153884,-0.135533,0.362174,0.097059,-0.143139,0.205211,0.210223,0.067028,-0.005734,-0.070416,-0.230524,-0.121157,-0.242449,-0.151424,-0.054341,0.366460,-0.191183,0.370701,0.010451,0.379251,-0.175467,-0.183683,0.067809,0.389908,0.038535,0.056936
koebner_phen,0.000498,-0.010024,0.239187,0.271582,1.000000,0.393592,-0.173438,0.392060,-0.065678,0.014118,-0.084046,0.385167,-0.047053,-0.170882,-0.248425,0.149341,-0.054368,0.001301,-0.039630,0.007275,-0.165733,0.024919,-0.002912,0.156883,0.399907,0.003301,0.375926,-0.002478,0.381149,-0.148507,-0.164937,0.081283,0.382270,0.084135,-0.092382
poly_papules,0.033449,-0.074691,0.326968,0.412835,0.393592,1.000000,-0.137526,0.865064,-0.275696,-0.254163,-0.152404,0.906988,0.028402,-0.315667,-0.158634,0.382805,0.129329,-0.165411,-0.044038,-0.295914,-0.402164,-0.288491,-0.207214,-0.202633,0.880901,-0.120473,0.911572,0.082789,0.895043,-0.095809,-0.110547,0.227406,0.905763,0.098149,0.057176
fol_papules,-0.109647,-0.092188,-0.168728,-0.144099,-0.173438,-0.137526,1.000000,-0.132810,0.222643,-0.003997,0.192623,-0.136595,-0.087748,-0.120354,0.019389,-0.009714,-0.096554,0.188790,-0.030869,-0.116437,-0.106159,-0.128934,-0.057658,-0.096282,-0.119131,-0.157808,-0.140404,-0.012057,-0.139582,0.785247,0.844903,-0.085999,-0.134335,-0.367499,0.477632
oral_muc_inv,-0.026946,-0.082639,0.285790,0.362073,0.392060,0.865064,-0.132810,1.000000,-0.283119,-0.260716,-0.152368,0.869156,0.013707,-0.304841,-0.140605,0.363164,0.101670,-0.156190,-0.041126,-0.285766,-0.388372,-0.278597,-0.200107,-0.191569,0.884285,-0.107444,0.887487,0.123183,0.875136,-0.082916,-0.106756,0.250015,0.892277,0.113243,0.055215
knee_elbow_inv,0.141151,0.298182,0.305436,-0.302110,-0.065678,-0.275696,0.222643,-0.283119,1.000000,0.660260,0.342462,-0.271951,-0.198378,0.332706,-0.227100,-0.533249,0.116670,0.230490,0.442761,0.722823,0.497589,0.645798,0.447940,0.518942,-0.267144,0.482614,-0.281784,-0.417120,-0.280029,0.233218,0.257988,-0.067483,-0.276325,-0.075976,-0.382977
scalp_inv,0.186609,0.302209,0.267366,-0.153884,0.014118,-0.254163,-0.003997,-0.260716,0.660260,1.000000,0.292321,-0.257704,-0.077537,0.352404,-0.227189,-0.531130,0.158563,0.186144,0.480579,0.769369,0.555367,0.746793,0.464010,0.632690,-0.256475,0.488372,-0.253429,-0.419951,-0.258154,-0.052285,-0.017731,0.033354,-0.250965,0.043486,-0.534427
